# SEIR model statistical prediction 

# Acknowledgements

based off code from: 

( Joshua McGee )
https://www.mathworks.com/matlabcentral/fileexchange/74676-fitviruscv19v3-covid-19-sir-model 

( Henri Frosse )
https://towardsdatascience.com/infectious-disease-modelling-beyond-the-basic-sir-model-216369c584c4

# imports

In [1]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# Constants and Relations

In [2]:
#Basics fill in with more accurate data and make program to calcuate from real data....

t = np.linspace(0, 100, 100) # Grid of time points (in days)
N = 1_600_000 # total popuation
Dur = 3.0 #days infective
gamma = 1.0 / Dur # recovery ratio
delta = 1.0 / 5.0  # 1 / incubation period 
alpha = 0.2  # death rate 
rho = 1/5  # 1/ # days from infection until death (death rate)
L= 80 # # days from start of outbreak to lockdown

#time / action dependent conditions
# to make more useful add more time depenedent series / turn this whol thing into a function

# simple version
# Number an infected person infects on avg given a lockdown L days from t0
# def R_0(t): 
#     return 5.0 if t < L else 0.9
# Avg # infected per day by a person, time depended due to R_0
# def beta(t): 
#     return R_0(t) * gamma

#logistic R0 change 

R_0_start, k, x0, R_0_end = 5.0, 0.5, 50, 0.5 
# intialR_0, rate of change r_0, infelction pt day #, endvalue R_0

def logistic_R_0(t):
    return (R_0_start-R_0_end) / (1 + np.exp(-k*(-t+x0))) + R_0_end

def beta(t):
    return logistic_R_0(t) * gamma


# Functions

In [3]:
# models state transitions as derivatives of change.
# notation change in number in state = rate in *prob * pop -rate out *prob*pop
#build these flow diagram backwards is a good idea

#(S)useptible -> (E)xposed -> (I)nfected -> (R)ecovered
#                                       |-> (D)ead)

def deriv(y, t, N, beta, gamma, delta, alpha, rho):
    S, E, I, R, D = y
    dSdt = -beta(t) * S * I / N
    dEdt = beta(t) * S * I / N - delta * E
    dIdt = delta * E - (1 - alpha) * gamma * I - alpha * rho * I
    dRdt = (1 - alpha) * gamma * I
    dDdt = alpha * rho * I
    return dSdt, dEdt, dIdt, dRdt, dDdt


# plotting function (base)

In [4]:
def plotseird(t, S, E, I, R, D=None, L=None, R0=None, Alpha=None):
    
  f, ax = plt.subplots(1,1,figsize=(10,4))

    # push each plot
  #ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
  ax.plot(t, E, 'y', alpha=0.7, linewidth=2, label='Exposed')
  ax.plot(t, I, 'r', alpha=0.7, linewidth=2, label='Infected')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')

  if D is not None:
    ax.plot(t, D, 'k', alpha=0.7, linewidth=2, label='Dead')
    #ax.plot(t, S+E+I+R+D, 'c--', alpha=0.7, linewidth=2, label='Total')
 # else:
   # ax.plot(t, S+E+I+R, 'c--', alpha=0.7, linewidth=2, label='Total')

  ax.set_xlabel('Time (days)')

  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend(borderpad=2.0)
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  if L is not None:
      plt.title("Lockdown after {} days".format(L))
  plt.show();

  if R0 is not None:
    f = plt.figure(figsize=(12,4))
  
  if R0 is not None:
    # sp1
    ax1 = f.add_subplot(121)
    ax1.plot(t, R0, 'b--', alpha=0.7, linewidth=2, label='R_0')

    ax1.set_xlabel('Time (days)')
    ax1.title.set_text('R_0 over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax1.yaxis.set_tick_params(length=0)
    ax1.xaxis.set_tick_params(length=0)
    ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax1.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)

  if Alpha is not None:
    # sp2
    ax2 = f.add_subplot(122)
    ax2.plot(t, Alpha, 'r--', alpha=0.7, linewidth=2, label='alpha')

    ax2.set_xlabel('Time (days)')
    ax2.title.set_text('fatality rate over time')
    # ax.set_ylabel('Number (1000s)')
    # ax.set_ylim(0,1.2)
    ax2.yaxis.set_tick_params(length=0)
    ax2.xaxis.set_tick_params(length=0)
    ax2.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax2.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)

    plt.show();

In [5]:
#S0, E0, I0, R0, D0 = N-1, 1, 0, 0, 0  # initial conditions: one exposed
#y0 = S0, E0, I0, R0, D0 # Initial conditions vector

#ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta, alpha, rho))
#S, E, I, R, D = ret.T

# Plot

In [6]:
#t = np.linspace(0, 100, 100) # Grid of time points (in days)
# to plot R_0 over time: get function values if you want to see it 
#R0_over_time = [logistic_R_0(i) for i in range(len(t))]

#plotseird(t, S, E, I, R, D)